# Answer to Question 2
## We will find the direction of the house using the direction to the closest road, and which direction that road is facing
We clearly make an assumption here that houses sit perpendicularly to the closest road 

### First imports

In [77]:
import requests, pandas as pd, math, time
from shapely.geometry import LineString, Point, shape
from geopy.distance import geodesic
import math
import json


### Using microburbs reporting tool to get house addresses and their co-ordinates

In [78]:
# using provided microburbs api call to get information about properties
url = "https://www.microburbs.com.au/report_generator/api/suburb/properties"
params = {
    "suburb": "Belmont North"
}
headers = {
    "Authorization": "Bearer test",
    "Content-Type": "application/json"
}

response = requests.get(url, params=params, headers=headers)
data = response.json()
print(data)

{'results': [{'address': {'sa1': '11101120615', 'sal': 'Belmont North', 'state': 'NSW', 'street': '25 Seacroft Close'}, 'area_level': 'address', 'area_name': '25 Seacroft Close, Belmont North, NSW', 'attributes': {'bathrooms': 2.0, 'bedrooms': 4.0, 'building_size': 'None', 'description': "Positioned on an expansive block with lush bushland to the rear, this incredible property creates a private sanctuary accentuating effortless family living, complemented by multiple living spaces and fabulous resort-style entertaining.\n\nFramed by verdant views and flooded with natural light, open-plan living acts an enticing hub, showcasing richly toned timber floors and polished design. Offering distinct zones for living and dining, the space is centred by a gorgeous kitchen, where keen home cooks will appreciate gas cooking and quality appliances. With three bedrooms and a tastefully appointed bathroom set off to the side, the fourth bedroom and bathroom is positioned downstairs, alongside a handy

### Data cleaning
Here we want to reduce the information to just maintain address, and state and co-ordinates as that is all we need

In [79]:
records = [] # using a simple list
for item in data["results"]: # iterate over all items

    addr = item["address"]
    coords = item["coordinates"]
    full_address = f"{addr['street']}, {addr['sal']}, {addr['state']}"

    records.append({
        "address": full_address,
        "state": addr["state"],
        "coordinates": f"({coords['latitude']}, {coords['longitude']})"
    })


# Create DataFrame
df = pd.DataFrame(records)

# Display
df.head(10)

,address,state,coordinates
0,"25 Seacroft Close, Belmont North, NSW",NSW,"(-33.01183148, 151.67286749)"
1,"67 Old Belmont Road, Belmont North, NSW",NSW,"(-33.01649474, 151.67157968)"
2,"17 Dulungra Avenue, Belmont North, NSW",NSW,"(-33.01204354, 151.66851581)"
3,"5 Pinnaroo Close, Belmont North, NSW",NSW,"(-33.02006063, 151.6800542)"
4,"30 John Fisher Road, Belmont North, NSW",NSW,"(-33.02311367, 151.66921469)"
5,"Unit 2, 1 Vincent Street, Belmont North, NSW",NSW,"(-33.02289234, 151.66747673)"
6,"46 Patrick Street, Belmont North, NSW",NSW,"(-33.02379398, 151.66499999)"
7,"380-384 Pacific Highway, Belmont North, NSW",NSW,"(-33.02305765, 151.66381854)"


### We need to parse co-ordinates and then apply a distance metric to match the distance from closest point on a road to the house

In [80]:
# --- Coordinate parsing ---
def parse_coords(coord_str):
    """Convert string coordinate to floats (lat, lon)."""
    if isinstance(coord_str, tuple) or isinstance(coord_str, list):
        return float(coord_str[0]), float(coord_str[1])
    nums = re.findall(r"-?\d+\.\d+", coord_str)
    if len(nums) == 2:
        return float(nums[0]), float(nums[1])
    else:
        raise ValueError(f"Invalid coordinate format: {coord_str}")

# --- Haversine distance ---
def haversine(lat1, lon1, lat2, lon2):
    R = 6371e3  # Earth radius in meters
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2)**2
    return R * (2 * math.atan2(math.sqrt(a), math.sqrt(1 - a)))






### Calculating the bearing from the closest point on road to house

In [81]:
def calculate_bearing(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    x = math.sin(dlon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(dlon)
    bearing = math.degrees(math.atan2(x, y))
    return (bearing + 360) % 360

### Finding the closest road & closest point on the road

In [82]:

# --- Query NSW Roads API ---
def find_closest_road(lat, lon, radius=50):
    url = "https://portal.spatial.nsw.gov.au/server/rest/services/NSW_Transport_Theme/MapServer/5/query"
    params = {
        "geometry": f"{lon},{lat}",
        "geometryType": "esriGeometryPoint",
        "inSR": 4326,
        "spatialRel": "esriSpatialRelIntersects",
        "distance": radius,
        "units": "esriSRUnit_Meter",
        "outFields": "roadnamebase,roadnametype,roadnamesuffix",
        "returnGeometry": "true",
        "f": "json"
    }
    resp = requests.get(url, params=params)
    data = resp.json()
    if "features" not in data or not data["features"]:
        return None, None, None, None

    feature = data["features"][0]
    attrs = feature["attributes"]
    road_name = " ".join(filter(None, [attrs.get("roadnamebase"), attrs.get("roadnametype"), attrs.get("roadnamesuffix")])).strip()

    # Get first vertex of road polyline
    geom = feature["geometry"]["paths"][0][0]
    road_lon, road_lat = geom[0], geom[1]

    dist = haversine(lat, lon, road_lat, road_lon)
    bearing = calculate_bearing(lat, lon, road_lat, road_lon)

    return road_name, road_lat, road_lon, dist, bearing

### Adding a direction column to the data frame

In [83]:
def add_direction_column(df):
    results = []
    for _, row in df.iterrows():
        lat, lon = parse_coords(row["coordinates"])
        _, _, _, _, bearing = find_closest_road(lat, lon)
        results.append(bearing)
    df["direction"] = results
    return df


df = add_direction_column(df)

print(df)

                                        address state  \
0         25 Seacroft Close, Belmont North, NSW   NSW   
1       67 Old Belmont Road, Belmont North, NSW   NSW   
2        17 Dulungra Avenue, Belmont North, NSW   NSW   
3          5 Pinnaroo Close, Belmont North, NSW   NSW   
4       30 John Fisher Road, Belmont North, NSW   NSW   
5  Unit 2, 1 Vincent Street, Belmont North, NSW   NSW   
6         46 Patrick Street, Belmont North, NSW   NSW   
7   380-384 Pacific Highway, Belmont North, NSW   NSW   

                    coordinates   direction  
0  (-33.01183148, 151.67286749)  358.627204  
1  (-33.01649474, 151.67157968)   51.049202  
2  (-33.01204354, 151.66851581)  168.147571  
3   (-33.02006063, 151.6800542)  251.259918  
4  (-33.02311367, 151.66921469)  195.429966  
5  (-33.02289234, 151.66747673)   16.762866  
6  (-33.02379398, 151.66499999)   96.894083  
7  (-33.02305765, 151.66381854)   67.719740  


In [84]:
df_output = df[['address', 'direction']].copy()

# Save to CSV
output_file = "address_direction.csv"
df_output.to_csv(output_file, index=False)

print(f"CSV saved to {output_file}")
print(df_output)

CSV saved to address_direction.csv
                                        address   direction
0         25 Seacroft Close, Belmont North, NSW  358.627204
1       67 Old Belmont Road, Belmont North, NSW   51.049202
2        17 Dulungra Avenue, Belmont North, NSW  168.147571
3          5 Pinnaroo Close, Belmont North, NSW  251.259918
4       30 John Fisher Road, Belmont North, NSW  195.429966
5  Unit 2, 1 Vincent Street, Belmont North, NSW   16.762866
6         46 Patrick Street, Belmont North, NSW   96.894083
7   380-384 Pacific Highway, Belmont North, NSW   67.719740
